# Aspect-Based Sentiment Analysis (ABSA) - Modular Version

This notebook demonstrates the ABSA functionality that has been organized into modular components:

- **`absa_core.py`**: Core ABSA algorithm
- **`setup.py`**: Dependency installation and model setup
- **`test_examples.py`**: Test cases and examples
- **`main.py`**: Main interface and interactive mode

## Usage Options:

1. **Run the notebook cells below** (original approach)
2. **Use the modular Python files**:
   ```bash
   python main.py  # For interactive mode with all examples
   ```
3. **Import as modules** in your own code:
   ```python
   from main import ABSAAnalyzer
   analyzer = ABSAAnalyzer()
   results = analyzer.analyze("Your text here")
   ```

In [ ]:
#Importing and Dowloading Libraries
!pip install stanza
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import stanza
from nltk.sentiment.vader import SentimentIntensityAnalyzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 47.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
def aspect_sentiment_analysis(txt, stop_words, nlp, sid):
    txt = txt.lower()
    sentList = nltk.sent_tokenize(txt)

    finalcluster = []

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList)
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        doc = nlp(finaltxt)

        dep_node = []
        for sent in doc.sentences:
            for dep_edge in sent.dependencies:
                dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i))
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            finalcluster.append([i[0], filist])

    # Sentiment analysis
    aspect_sentiments = []
    for aspect in finalcluster:
        aspect_text = aspect[0]
        sentiment_score = sid.polarity_scores(aspect_text)['compound']
        aspect_sentiments.append([aspect_text, sentiment_score])

    return aspect_sentiments

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline()


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
sid = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))

In [ ]:
txt = "The battery life of this phone is excellent, but the camera quality is disappointing."
print(aspect_sentiment_analysis(txt, stop_words, nlp, sid))

[['batterylife', 0.0], ['phone', 0.0], ['excellent', 0.5719], ['cameraquality', 0.0], ['disappointing', -0.4939]]


In [ ]:
txt1 = "Shreyas is intelligent but he is very lazy."
print(aspect_sentiment_analysis(txt1,stop_words,nlp,sid))

[['shreyas', 0.0], ['intelligent', 0.4588], ['lazy', -0.3612]]


In [ ]:
txt2 = "The ice cream is great but the waffle is hard to eat."
print(aspect_sentiment_analysis(txt2,stop_words,nlp,sid))

[['icecream', 0.0], ['great', 0.6249], ['waffle', 0.0], ['hard', -0.1027], ['eat', 0.0]]


In [ ]:
txt01="Quality of the dress is good but the colour is dull"
print(aspect_sentiment_analysis(txt01,stop_words,nlp,sid))

[['quality', 0.0], ['dress', 0.0], ['good', 0.4404], ['colour', 0.0], ['dull', -0.4019]]


In [ ]:
txt02="Appearance of the cover page is beautiful but paper quality is poor"
print(aspect_sentiment_analysis(txt02,stop_words,nlp,sid))

[['appearance', 0.0], ['coverpage', 0.0], ['beautiful', 0.5994], ['paperquality', 0.0], ['poor', -0.4767]]


In [ ]:
t="The movie was super but the screen and sound quality in the theatre were horrible"
print(aspect_sentiment_analysis(t,stop_words,nlp,sid))

[['movie', 0.0], ['super', 0.5994], ['screen', 0.0], ['sound', 0.0], ['quality', 0.0], ['theatre', 0.0], ['horrible', -0.5423]]


In [ ]:
s = 'i am an bad guy but also good at the same time'
print(aspect_sentiment_analysis(s,stop_words, nlp, sid))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
txt = "The battery life of this phone is excellent, but the camera quality is\xa0disappointing."
print(aspect_sentiment_analysis(txt, stop_words, nlp, sid))

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
